### Import Required Libraries and Set Up Environment Variables

In [8]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [9]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [10]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
#query_url = url + "begin_date=" + begin_date + "&end_date=" + end_date + "fq=" + filter_query + "&field-name:" + field_list + "&sort=" + sort  "&api-key=" + nyt_api_key 

query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
    )

In [12]:
# Create an empty list to store the reviews
reviews_list=[]

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    page_query = f"{query_url}&page={str(page)}"
    
    # API results show 10 articles at a time  
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_query).json()
    #reviews_json = reviews.json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:         
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:   
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Checked page {page}")         

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page {page} had no results")
        pass
        

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [127]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [128]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [138]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

reviews_df['title']


<bound method Series.isnull of 0                             The Attachment Diaries
1      What’s Love Got to Do With It?’ Probably a Lo
2                               You Can Live Forever
3                          A Tourist’s Guide to Love
4                            Other People’s Children
                           ...                      
195                                   The Other Half
196                           The Ottoman Lieutenant
197                                     Love & Taxes
198    Everybody Loves Somebody,’ a Rom-Com With Bit
199                Kedi,’ Rekindling a ‘Love of Life
Name: title, Length: 200, dtype: object>

In [132]:
reviews_df['keywords']


0      [{'name': 'subject', 'value': 'Movies', 'rank'...
1      [{'name': 'subject', 'value': 'Movies', 'rank'...
2      [{'name': 'subject', 'value': 'Movies', 'rank'...
3      [{'name': 'subject', 'value': 'Movies', 'rank'...
4      [{'name': 'subject', 'value': 'Movies', 'rank'...
                             ...                        
195    [{'name': 'subject', 'value': 'Movies', 'rank'...
196    [{'name': 'subject', 'value': 'Movies', 'rank'...
197    [{'name': 'creative_works', 'value': 'Love & T...
198    [{'name': 'subject', 'value': 'Movies', 'rank'...
199    [{'name': 'creative_works', 'value': 'Kedi (Mo...
Name: keywords, Length: 200, dtype: object

In [124]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)

reviews_df['keywords']

TypeError: string indices must be integers

In [147]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_df['title'].to_list()
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [140]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key



In [149]:
# Create an empty list to store the results
tmdb_movie_list=[]


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Create containers to hold aditional movie detail data
genres = []
spoken_languages = []
production_countries = []

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    # Add 1 to the request counter
    request_counter += request_counter

    if (request_counter % 50) == 0:
        time.sleep(1)
        print("Application sleeping")

    
    # Perform a "GET" request for The Movie Database
    request_url = url + str(title) + tmdb_key_string
    response_data = requests.get(request_url).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        #QUESTION - ITERATE THROUGH RESULTS LIST?
        # Get movie id 
        movie_id = response_data["results"][0]["id"]

        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"

        # Execute "GET" request with url
        movie_details = requests.get(details_url).json()
         #print(json.dumps(movie_details, indent=4))
        
        # Extract the genre names into a list
        genres = response_data["results"][0]["id"]
        
        # Extract the spoken_languages' English name into a list
        spoken_languages.append(response_data['results']['name'])
        
        # Extract the production_countries' name into a list        
        production_countries.append(response_data[0]['results']['rating']['average'])


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        movie_dict = {
                    "title": response_data["title"],
                    "original_title": response_data["original_title"],
                    "budget": response_data["budget"],
                    "original_language": response_data["original_language"],
                    "homepage": response_data["homepage"],
                    "overview": response_data["overview"],
                    "popularity": response_data["popularity"],
                    "runtime": response_data["runtime"],
                    "revenue": response_data["revenue"],
                    "release_date": response_data["release_date"],
                    "vote_average": response_data["vote_average"],
                    "vote_count": response_data["vote_count"],
                    "genres": genres,
                    "spoken_languages": spoken_languages,
                    "production_countries": production_countries
                }
                    
        tmdb_movie_list.append(movie_dict)  
       
        # Print out the title that was found
        Print(f"Found {title}")
    except:
        print(f"{title} not found")
        pass

The Attachment Diaries not found
What’s Love Got to Do With It?’ Probably a Lo not found
You Can Live Forever not found
A Tourist’s Guide to Love not found
Other People’s Children not found
One True Loves not found
The Lost Weekend: A Love Story not found
A Thousand and One not found
Your Place or Mine not found
Love in the Time of Fentanyl not found
Pamela, a Love Story not found
In From the Side not found
After Love not found
Alcarràs not found
Nelly & Nadine not found
Lady Chatterley’s Lover not found
The Sound of Christmas not found
The Inspection not found
Bones and All not found
My Policeman not found
About Fate not found
Waiting for Bojangles not found
I Love My Dad not found
A Love Song not found
Alone Together not found
Art of Love not found
The Wheel not found
Thor: Love and Thunder not found
Both Sides of the Blade not found
Fire of Love not found
Love & Gelato not found
Stay Prayed Up not found
Benediction not found
Dinner in America not found
In a New York Minute not found

In [154]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(response_data[:5], indent=4))

{'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}


TypeError: unhashable type: 'slice'

In [13]:
# Convert the results to a DataFrame
movie_df = pd.json_normalize(tmdb_movie_list)
movie_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,0,2021-10-07,4.000,2,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,0,2023-03-24,7.643,21,[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy, Adventure]",en,[English],https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.369,96,0,2023-04-21,6.389,122,[United States of America]
3,Other People's Children,Les Enfants des autres,0,[Drama],fr,[French],https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",16.246,104,0,2022-09-21,6.900,137,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,37820,2023-04-07,6.681,36,"[Germany, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,0,2017-01-11,5.617,184,[United States of America]
151,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,0,2017-01-14,6.051,108,"[Belgium, France]"
152,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,0,2006-05-15,6.708,12,[United Kingdom]
153,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[English, Turkish]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",12.097,111,413844,2017-03-10,6.150,236,"[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = movie_pd.merge(reviews_df, movies_df, on = 'title')
merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['col_1', 'col_1', 'col_1', 'col_1']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for i, column in enumerate(columns_to_fix):
    
    # Convert the column to type 'str'
    columns_to_fix[i] = str(column)

    # Loop through characters to remove
    str.replace(characters_to_remove)

# Display the fixed DataFrame
merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [16]:
# Drop "byline.person" column
merged_df.drop("byline.person", axis=1)

In [17]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [18]:
# Export data to CSV without the index
merged_df.to_csv('merged_movies.csv', index=False)